# Funciones de pérdida

La función de pérdida es un componente fundamental del proceso de entrenamiento de una RN. Mide la distancia entre la salida obtenida por el modelo (para una determinada entrada) y la salida deseada. Este valor es utilizado por el optimizador del modelo para realizar el cálculo de los gradientes del modelo, al final de cada *step* del proceso de entrenamiento.

Existen dos categorías de funciones de pérdida:

- Regresión: Son útiles para procesos de optimización cuyo objetivo es predecir un valor numérico en un intervalo continuo (ej. la probabilidad de lluvia).
- Clasificación: Son útiles para tareas en las que el objetivo es predecir un valor entre varios posibles valores categóricos (ej. la especie de un animal que aparece en una fotografía).

En Pytorch, existen implementaciones para varias funciones de optimización populares. Antes de ver algunos ejemplos, veamos cómo podemos realizar el cálculo de una función de pérdida sencilla utilizando operaciones básicas sobre tensores.




In [1]:
import torch

# Genera dos salidas de la red neuronal sintéticas
output1 = torch.tensor([1.0, 2.0, 3.0, 4.0])  # Sample output 1
output2 = torch.tensor([0.5, 1.5, 2.5, 3.5])  # Sample output 2

# Genera un vector de valores objetivo (ground truth)
target = torch.tensor([0.8, 1.7, 2.9, 3.8])

# Calcula la diferencia al cuadrado entre las salidas y los valores objetivo
squared_diff1 = torch.pow(output1 - target, 2)
squared_diff2 = torch.pow(output2 - target, 2)

# Calcula el error cuadrático medio (MSE) entre las salidas y los valores objetivo
loss = torch.mean(squared_diff1) + torch.mean(squared_diff2)
print("MSE Loss:", loss.item())


MSE Loss: 0.14000000059604645


Exite una función en Pytorch que implementa el MSE, la cual se puede utilizar de la siguiente manera:

In [5]:
import torch
output1 = torch.tensor([1.0, 2.0, 3.0, 4.0])  # Salida de ejemplo 1
output2 = torch.tensor([0.5, 1.5, 2.5, 3.5])  # Salida de ejemplo 2
target = torch.tensor([0.8, 1.7, 2.9, 3.8])

# Calcula el error cuadrático medio (MSE) entre las salidas y los valores objetivo usando la función MSELoss
criterion = torch.nn.MSELoss()
loss = criterion(output1, target) + criterion(output2, target)
print("MSE Loss:", loss.item())


MSE Loss: 0.14000000059604645


De aquí en adelante vamos a usar ejemplos en los que siempre se usan funciones que implementan cálculos de funciones de pérdida populares en Pytorch. 

Siguiendo, dentro de la categoría de funciones de pérdida regresivas, vemos ahora un ejemplo, basado en el miso caso de uso de la función de pérdida L1Loss

In [7]:
# Calcula el error cuadrático medio (L1) entre las salidas y los valores objetivo usando la función L1Loss
criterion = torch.nn.L1Loss()
loss = criterion(output1, target) + criterion(output2, target)
print("L1 Loss:", loss.item())

L1 Loss: 0.5


Vemos que en este caso el valor de pérdida obtenido es mucho más alto. La diferencia es que MSELoss usaba el cuadrado de las diferencias, mientras L1Loss utiliza para su cálculo el valor absoluto de la diferencia.

Veamos ahora, una reproducción del mismo ejemplo, utilizando la función de pérdida regresiva Mean Bias Error (MBE). En este caso, no hay ninguna implementación para Pytorch, así que utilizamos una implementación hecha por nosotros mismos.

In [11]:
# Calcula el Mean Bias Error (MBE) entre las salidas y los valores objetivo 
diff1 = output1 - target
diff2 = output2 - target
print(diff1)
print(diff2)
loss = torch.mean(diff1) + torch.mean(diff2)
print("MBE Loss:", loss.item()/2)

tensor([0.2000, 0.3000, 0.1000, 0.2000])
tensor([-0.3000, -0.2000, -0.4000, -0.3000])
MBE Loss: -0.05000001937150955


# Actividad

Dadas las salidas reales y desadas anteriores: *outpu1*, *output2*, *target*, calcula una función de pérdida con la siguiente formulación.

FPP = ∑ |output1 - target| + ∑ |output2 - target|


In [ ]:
#Pon tu código aquí

## Funciones de pérdida categóricas

Vamos a explorar ahora varios ejemplos de funciones de pérdida categóricas. En este caso, se abordan problemas en los que la salida del modelo es un vector de probabilidades. Cada elemento de dicho vector, mide la probabilidad de que la entrada se clasifique en la categoría correspondiente. 

Un ejemplo sencillo, sería una RN para la cual se predice el número contenido en imágenes del conjunto de datos CIFAR-10. La salida es un vector de 10 elementos, donde el primer elemento contiene la probabilidad de que el número en la imagen sea un 0, el segundo elemento la probabilidad de que sea un 1, etc...

Las funciones de pérdida de este tipo deben capturar la diferencia entre la clase prevista y las probabilidades contenidas en vector de salida obtenido de la inferencia del modelo. La clase prevista asigna un 100% de probabilidad a la clase correcta y 0 a todas las demás, por lo tanto, las funciones de pérdida deben capturar cuánto se alejan las probabilidades del modelo de esta predicción perfecta.

Las funciones de pérdida categóricas tratan de capturar esta diferencia. Una de ellas es *SVM Loss* y vemos ahora un ejemplo sobre esta función cuya fórmula simplificada es SVM Loss = ∑ max(0, 1 - y_i * f(x_i))


In [1]:
import torch

# Predicción y objetivo de ejemplo
predictions = torch.tensor([0.85, 0.12, 0.08, 0.01, 0, 0, 0.23, 0, 0.4 ,0.2]) 
targets = torch.tensor([1, 0, 0, 0, 0, 0, 0, 0, 0 ,0])

# Cálculo de la pérdida SVM
loss = torch.mean(torch.max(torch.zeros_like(targets), 1 - targets * predictions))
print("SVM Loss:", loss.item())


SVM Loss: 2.7039999961853027


La función de Pytorch *MultiLabelMaginLoss* proporciona una implementación de esta métrica en Pytorch. Veamos el mismo ejemplo usando esta función.

In [4]:
# Cálculo de la pérdida SVM con Pytorch
loss = torch.nn.MultiLabelMarginLoss()
loss = loss(predictions.unsqueeze(0), targets.unsqueeze(0))
print("SVM Loss:", loss.item())

SVM Loss: 2.7039999961853027


Otro ejemplo de función de pérdida categórica es Cross Entropy Likelihood Loss. Es una de las más populares y se implementa en Pytorch con la función *torch.nn.CrossEntropyLoss*. Veamos el mismo ejemplo, usando esta función.


In [6]:
# Cálculo de la pérdida CEL
loss = torch.nn.CrossEntropyLoss()
loss = loss(predictions.unsqueeze(0), targets.argmax().unsqueeze(0))
print("CEL Loss:", loss.item())

CEL Loss: 1.6783099174499512


# Ejercicio

Revisa los contenidos del código que se muestra a continuación y prueba alternativamente a utilizar loss_function1, loss_function2 y loss_function3 como funciones de pérdida. 

Para ello, se tiene que cambiar la línea de código

*loss = loss_func1(outputs, labels)*

por la otra versión
y ajustar la forma de procesar la capa de salida en la pasada *forward*. Ver código comentado.

¿Observas diferencias en su eficacia?

In [8]:
# Import libraries
import torch
import torchvision
import torchvision.transforms as transforms

# Download and prepare the CIFAR-10 dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 
           'dog', 'frog', 'horse', 'ship', 'truck')

# Create a neural network model
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # Capa de salida cuando usamos CrossEntropyLoss
        x = self.fc3(x)
        # Capa de salida cuando usamos NLLLoss
        # x=F.log_softmax(self.fc3(x), dim=1)
        return x

# Initialize the model
net = Net()
# Define two loss functions
loss_func1 = nn.CrossEntropyLoss()
loss_func2 = nn.NLLLoss()

# Define an optimizer
import torch.optim as optim
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Training the network using CrossEntropyLoss
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = loss_func1(outputs, labels)
        # loss = loss_func2(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')


Files already downloaded and verified
Files already downloaded and verified
[1,  2000] loss: 2.192
[1,  4000] loss: 1.895
[1,  6000] loss: 1.710
[1,  8000] loss: 1.600
[1, 10000] loss: 1.527
[1, 12000] loss: 1.481
[2,  2000] loss: 1.413
[2,  4000] loss: 1.369
[2,  6000] loss: 1.380
[2,  8000] loss: 1.324
[2, 10000] loss: 1.318
[2, 12000] loss: 1.287
Finished Training
